In [2]:
import pandas as pd
import numpy as np

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import BaselineOnly, SVDpp, SVD

In [53]:
df_collab = pd.read_csv('./data/review_allvid_clean.csv')
df_collab.head()

,rating,user_id,movie_id,reviews
0,5,A2VHSG6TZHU1OB,0001527665,Having lived in West New Guinea (Papua) during...
1,5,A1KM9FNEJ8Q171,0001527665,"More than anything, I've been challenged to fi..."
2,4,A38LY2SSHVHRYB,0001527665,This is a great movie for a missionary going i...
3,5,AHTYUW2H1276L,0001527665,This movie was in ENGLISH....it was a great su...
4,5,A3M3HCZLXW0YLF,0001527665,"This is a fascinating true story, well acted b..."


In [54]:
df_col = df_collab.copy()

In [55]:
df_col.drop(columns='reviews', axis=1, inplace=True)
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6224034 entries, 0 to 6224033
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
dtypes: int64(1), object(2)
memory usage: 142.5+ MB


In [5]:
#df_col.drop_duplicates(subset=['user_id', 'movie_id'], keep='first', inplace=True)

In [6]:
#df = df_col[df_col['user_id'].isin(df_col['user_id'].value_counts()[df_col['user_id'].value_counts() >= 5].index)]
#df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2170362 entries, 9 to 6224032
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
dtypes: int64(1), object(2)
memory usage: 66.2+ MB


In [56]:
df = df_col.copy()

In [7]:
df['user_id'].value_counts().tail()

user_id
A3ELQQQZH907O8    5
AWC7HXAAVNEAR     5
A1PJO0NTSORTRM    5
A1SCGKAXSQ19U3    5
A2HLF4KO1QMO5B    5
Name: count, dtype: int64

In [60]:
df['movie_id'].nunique()

127613

In [57]:
movies_df = pd.read_csv('./data/meta_allvid_clean.csv')
movies_df.drop(columns =['english', 'rank'], inplace=True)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95076 entries, 0 to 95075
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        94610 non-null  object
 1   description  86140 non-null  object
 2   title        95061 non-null  object
 3   starring     95076 non-null  object
 4   movie_id     95076 non-null  object
dtypes: object(5)
memory usage: 3.6+ MB


In [58]:
all_vid = df['movie_id'].unique().tolist()
df_movies = movies_df[movies_df['movie_id'].isin(all_vid)]
df_movies.dropna(subset='genre', inplace=True)
df_movies.dropna(subset='title', inplace=True)
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94595 entries, 0 to 95075
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        94595 non-null  object
 1   description  85716 non-null  object
 2   title        94595 non-null  object
 3   starring     94595 non-null  object
 4   movie_id     94595 non-null  object
dtypes: object(5)
memory usage: 4.3+ MB


In [76]:
all_vid2 = df_movies['movie_id'].unique().tolist()
df2 = df[df['movie_id'].isin(all_vid2)]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6031186 entries, 7 to 6224033
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
dtypes: int64(1), object(2)
memory usage: 184.1+ MB


In [77]:
#df2.drop_duplicates(subset=['user_id', 'movie_id'], keep='first', inplace=True)
df2 = df2[df2['user_id'].isin(df2['user_id'].value_counts()[df2['user_id'].value_counts() >= 5].index)]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2128950 entries, 9 to 6224032
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
dtypes: int64(1), object(2)
memory usage: 65.0+ MB


In [86]:
df2['movie_id'].nunique()

94199

In [88]:
df_movies['movie_id'].nunique()

94595

In [ ]:
# <class 'pandas.core.frame.DataFrame'>
# Index: 2170362 entries, 9 to 6224032
# Data columns (total 3 columns):
#  #   Column    Dtype 
# ---  ------    ----- 
#  0   rating    int64 
#  1   user_id   object
#  2   movie_id  object
# dtypes: int64(1), object(2)
# memory usage: 66.2+ MB

In [65]:
reader = Reader()
data = Dataset.load_from_df(df2[['user_id', 'movie_id', 'rating']], reader)

In [66]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [67]:
baselinee = BaselineOnly()
baselinee.fit(trainset)
predictions = baselinee.test(testset)
base_pred = accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.9258


In [8]:
svd_cv = SVD()
cv_svd = cross_validate(svd_cv, data, measures=['RMSE'], n_jobs=-1, verbose=True)

for i in cv_svd.items():
    print(i)
print('-----------------------')
print(np.mean(cv_svd['test_rmse']))

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9168  0.9179  0.9175  0.9183  0.9205  0.9182  0.0012  
Fit time          13.18   13.45   13.56   13.23   12.91   13.26   0.22    
Test time         3.76    3.70    3.40    3.24    3.02    3.42    0.28    
('test_rmse', array([0.91682687, 0.9178684 , 0.91753398, 0.91826024, 0.92046403]))
('fit_time', (13.17979097366333, 13.446223020553589, 13.556465148925781, 13.225394010543823, 12.910300493240356))
('test_time', (3.764014482498169, 3.6977412700653076, 3.3994007110595703, 3.237215042114258, 3.0191333293914795))
-----------------------
0.9181907022099839


In [9]:
params = {'n_factors': [10, 20, 50, 100],
          'n_epochs':[10, 20, 40, 60]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1, joblib_verbose=10)
g_s_svd.fit(data)

print(g_s_svd.best_score)
print(g_s_svd.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  55 out of  65 | elapsed:  1.7min remaining:   18.7s
[Parallel(n_jobs=-1)]: Done  62 out of  65 | elapsed:  1.7min remaining:    4.9s


KeyboardInterrupt: 

In [ ]:
# params = {'n_factors': [10, 20, 50, 100],
#           'n_epochs':[10, 20, 40, 60],
#          'reg_all': [0.01, 0.02, 0.05, 0.1],
#          'lr_all': [0.002, 0.005, 0.01]}
# g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1, joblib_verbose=10)
# g_s_svd.fit(data)

# print(g_s_svd.best_score)
# print(g_s_svd.best_params)

In [10]:
SVD_base = SVD(n_factors=10, n_epochs=40, random_state=42)
SVD_base.fit(trainset)
predictions = SVD_base.test(testset)
kn_first = accuracy.rmse(predictions)

RMSE: 0.9120


In [11]:
svd_pp_cv = SVDpp()
cv_svdpp = cross_validate(svd_pp_cv, data, measures=['RMSE'], n_jobs=-1, verbose=True)

for i in cv_svdpp.items():
    print(i)
print('-----------------------')
print(np.mean(cv_svdpp['test_rmse']))

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9065  0.9110  0.9083  0.9098  0.9105  0.9092  0.0017  
Fit time          35.20   35.35   35.33   35.29   35.14   35.27   0.08    
Test time         10.04   10.01   10.02   9.92    9.85    9.97    0.07    
('test_rmse', array([0.90645347, 0.9109963 , 0.90828251, 0.90983874, 0.91046576]))
('fit_time', (35.20462965965271, 35.35350322723389, 35.33487033843994, 35.29214549064636, 35.14257192611694))
('test_time', (10.04360580444336, 10.00626540184021, 10.02463698387146, 9.91608214378357, 9.84733533859253))
-----------------------
0.9092073537848441


In [28]:
params = {'n_factors': [10, 20, 30, 50],
          'n_epochs': [10, 20, 30, 50, 100],
          'reg_all': [0.02, 0.05, 0.1],
          'cache_ratings': [True, False],
          'lr_all': [0.002, 0.005, 0.01],
          'verbose':[True]
         }
g_s_svdpp = GridSearchCV(SVDpp,param_grid=params,n_jobs=-1, measures=['RMSE'], joblib_verbose=10)
g_s_svdpp.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   39.4s


KeyboardInterrupt: 

In [ ]:
print(g_s_svdpp.best_score)
print(g_s_svdpp.best_params)

In [68]:
SVDpp_final = SVDpp()
SVDpp_final.fit(trainset)
predictions = SVDpp_final.test(testset)
kn_first = accuracy.rmse(predictions)


#output for user_id with >3 reviews = 1.0246

RMSE: 0.9071


In [19]:
#USE THIS FUNCTION TO CONFIRM THAT THE REC'S ARE MOVIES THAT HAVE NOT BEEN SEEN BY THE USER



# def recommend_movies(user_id, trained_model, movie_df, N=10):
#     # Get a list of all movies that the user hasn't seen yet
#     user_movies = movie_df[movie_df['user_id'] == user_id]['movie_id'].tolist()
#     all_movies = movie_df['movie_id'].tolist()
#     unseen_movies = set(all_movies) - set(user_movies)

#     # Create a dataframe of predictions for all unseen movies
#     predictions = []
#     for movie_id in unseen_movies:
#         predicted_rating = trained_model.predict(user_id, movie_id).est
#         predictions.append({'movie_id': movie_id, 'predicted_rating': predicted_rating})
#     predictions_df = pd.DataFrame(predictions)

#     # Sort predictions by rating and return top N
#     top_N = predictions_df.sort_values('predicted_rating', ascending=False).head(N)
#     top_N_movie_ids = top_N['movie_id'].tolist()

#     # Get the details of the top N movies, including title and rating
#     top_N_movies = movie_df[movie_df['movie_id'].isin(top_N_movie_ids)]
#     top_N_movies.drop_duplicates(subset=['title'], inplace=True)

#     # Merge top N movie ratings with the details dataframe
#     top_N_ratings = pd.merge(top_N, top_N_movies, on='movie_id')

#     # Return the top N movie details (excluding title)
#     return top_N_ratings.drop('title', axis=1)

In [79]:
def recommend_movies(user_id, trained_model, movie_df, N=10):
    # Get a list of all movies that the user hasn't seen yet
    user_movies = movie_df[movie_df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = movie_df['movie_id'].tolist()
    unseen_movies = set(all_movies) - set(user_movies)

    # Create a dataframe of predictions for all unseen movies
    predictions = []
    for movie_id in unseen_movies:
        predicted_rating = trained_model.predict(user_id, movie_id).est
        predictions.append({'movie_id': movie_id, 'predicted_rating': predicted_rating})
    predictions_df = pd.DataFrame(predictions)

    # Sort predictions by rating and return top N
    top_N = predictions_df.sort_values('predicted_rating', ascending=False).head(N)
    top_N_movie_ids = top_N['movie_id'].tolist()

    # Get the details of the top N movies, including title and rating
    top_N_movies = movie_df[movie_df['movie_id'].isin(top_N_movie_ids)]
    top_N_movies.drop_duplicates(subset=['movie_id'], inplace=True)

    # Merge top N movie ratings with the details dataframe
    top_N_ratings = pd.merge(top_N, top_N_movies, on='movie_id')

    # Return the top N movie details (excluding title)
    return top_N_ratings.drop(columns=['user_id', 'rating'], axis=1)

In [90]:
merged_df = pd.merge(df2, df_movies, on="movie_id", how="left")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128950 entries, 0 to 2128949
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   rating       int64 
 1   user_id      object
 2   movie_id     object
 3   genre        object
 4   description  object
 5   title        object
 6   starring     object
dtypes: int64(1), object(6)
memory usage: 113.7+ MB


In [91]:
merged_df['movie_id'].nunique()

94199

In [74]:
merged_df['title'].isnull().sum()

0

In [80]:
merged_df['user_id'].value_counts().tail(55)

user_id
A1Y393RU0D034C    5
A37WWPJPSC1TH0    5
A21QW4A2N4MY2J    5
APGUOMA4TXA8H     5
A2QDPJRVXL7P4F    5
A1KNSHHIZJ1WX8    5
A2U77U71F6PLX8    5
AHS2OHKZBIQL7     5
A1ZI18XD5UVUKB    5
A1T4UVC1ZDQE9D    5
AKRBCQ7GLSSL2     5
A26X4ACKD32R2O    5
AREE1B6VESH5A     5
A3KMUBDWJ92FMM    5
A27I9GINWSXPBW    5
A3SCJSUDQU0B9M    5
A1JP8YZBR95E8L    5
A2QV790TPL2BH5    5
A1QVOW9O6MQDXL    5
A200V67LXY8386    5
AFQWINGW8YHTC     5
A1SLRCDZXWU7BH    5
A3CRZI8WSBTYMH    5
A1B2LXLWNA5I8L    5
A3KPOPE1SAH8FN    5
A3TYOOXDY28OV8    5
A29H4NJGHQQJ3     5
A1X5COOJ796A84    5
AS14NV2O3HL1E     5
A1TWZER8QEZETA    5
A3F47M9AWV6HCG    5
AY0NMW3L1PHAC     5
A2A17LRHKN464Q    5
A30SII42AWOI10    5
A19MBSHIW2HWFY    5
A2OFV6HFZP750G    5
A3QOE23ZM9P415    5
A3KK1IO1W8YCNS    5
ALDXNZPGD6C6W     5
A3CN9X69R9MR06    5
A27SQYT9H4P4GE    5
A2OY8H75U4H0Y9    5
A3J0BZLWS6N2DI    5
A3617HBW5ECI77    5
A193XFQ5B6JEHZ    5
A1GUQJWB5ZBSJ     5
A4T6XG06B7SY7     5
A1NTNGBHPB7UJ7    5
A2RZGSJ41QDDT1    5
A21KTTVGI2S5

In [85]:
user_id = 'A1Y393RU0D034C'
recommendations = recommend_movies(user_id, SVDpp_final, merged_df, N=5)
recommendations

C:\Users\raxmo\AppData\Local\Temp\ipykernel_16964\954166125.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_N_movies.drop_duplicates(subset=['movie_id'], inplace=True)


,movie_id,predicted_rating,genre,description,title,starring
0,B00S1ITA2W,4.994772,All Fox Titles,Based upon the acclaimed comic book and direct...,Kingsman: The Secret Service,Various Artists
1,630356027X,4.973899,A&E Home Video All A&E Titles,They are the world's most celebrated bodyguard...,Secret Service VHS,Various Artists
2,0790729628,4.825320,Science Fiction & Fantasy Science Fiction Futu...,The Director's Cut dvd edition of Ridley Scott...,Blade Runner,Harrison Ford
3,B0029O0ANW,4.814167,HBO,"At first glance, Ray Drecker seems like an ord...",Hung: Season 1,Thomas Jane
4,B000O5GRXU,4.734966,Movies,Dvd,The Conscientious Objector,Terry Benedict


In [83]:
merged_df.loc[merged_df['user_id'] == 'A1Y393RU0D034C']

,rating,user_id,movie_id,genre,description,title,starring
338185,2,A1Y393RU0D034C,B00005JLTN,Science Fiction & Fantasy Science Fiction Futu...,Amid a wave of humans waking up from their 'Ma...,The Matrix Reloaded,Keanu Reeves
396911,2,A1Y393RU0D034C,B00006NT1R,HBO,"Professionally, the Fisher family owns a Los A...",Six Feet Under - The Complete First Season VHS,Peter Krause
1660613,5,A1Y393RU0D034C,B00UNEJ2LQ,Drama,An Israeli woman (Ronit Elkabetz) seeking to f...,Gett: The Trial of Viviane Amsalem,Simon Abkarian
1702924,4,A1Y393RU0D034C,B00YAZNBTI,Documentary,The events behind Kurt Cobains death as seen t...,Soaked In Bleach,Daniel Roebuck
1756774,4,A1Y393RU0D034C,B015DFI986,Drama,"In a gripping true story, American chess prodi...",Pawn Sacrifice,Tobey Maguire


In [92]:
merged_df.to_csv('./data/collab_model_df.csv', encoding='utf-8', index=False)